## _*FermionicOperator and qubit mapping*_

When we compute a FermionicOperator in Qiskit Chemistry it needs to be converted to a qubit operator to run on the simulator or real device. The FermionicOperator is built from electronn integrals where electrons behave anti-symmetrically under swap. qubits however do not exhibit this behavior and hence a mapping is needed to ensure that this is accounted for.

Here we have the jordan wigner mapping, the bravyi-kitaev mapping and a parity.

This notebook has been written to use the PYQUANTE chemistry driver. See the PYQUANTE chemistry driver readme if you need to install the external PyQuante2 library that this driver requires.

In [8]:
import numpy as np

from qiskit import BasicAer
from qiskit.transpiler import PassManager

from qiskit.aqua import Operator, QuantumInstance, aqua_globals
from qiskit.aqua.algorithms.adaptive import VQE
from qiskit.aqua.algorithms.classical import ExactEigensolver
from qiskit.aqua.components.optimizers import L_BFGS_B
from qiskit.aqua.components.variational_forms import RY

from qiskit.chemistry import FermionicOperator
from qiskit.chemistry.drivers import PyQuanteDriver, UnitsType, BasisType

aqua_globals.random_seed = 50

In [9]:
# using driver to get fermionic Hamiltonian
# PyQuante example
driver = PyQuanteDriver(atoms='H .0 .0 .0; H .0 .0 0.735', units=UnitsType.ANGSTROM,
                        charge=0, multiplicity=1, basis=BasisType.BSTO3G)
molecule = driver.run()
h1 = molecule.one_body_integrals
h2 = molecule.two_body_integrals

In [10]:
# convert from fermionic hamiltonian to qubit hamiltonian
ferOp = FermionicOperator(h1=h1, h2=h2)
qubitOp_jw = ferOp.mapping(map_type='JORDAN_WIGNER', threshold=0.00000001)
qubitOp_pa = ferOp.mapping(map_type='PARITY', threshold=0.00000001)
qubitOp_bk = ferOp.mapping(map_type='BRAVYI_KITAEV', threshold=0.00000001)

In [11]:
# print out qubit hamiltonian in Pauli terms and exact solution
qubit_ops = [(qubitOp_jw, 'jordan wigner'),
            (qubitOp_pa, 'parity'),
            (qubitOp_bk, 'bravyi-kitaev')]

for qubit_op, name in qubit_ops:
    qubit_op.to_matrix()
    qubit_op.chop(10**-10)

    print("\n --- {} ---".format(name))
    print(qubit_op.print_operators())

    # Using exact eigensolver to get the smallest eigenvalue
    exact_eigensolver = ExactEigensolver(qubit_op, k=1)
    ret = exact_eigensolver.run()
    print('The exact ground state energy using {} mapping is: {}'.format(name, ret['energy']))    


 --- jordan wigner ---
IIII	(-0.8105479862760991+0j)
IIIZ	(0.17218394273085635+0j)
IIZI	(-0.22575350251540605+0j)
IIZZ	(0.12091263358559995+0j)
IZII	(0.17218394273085635+0j)
IZIZ	(0.16892754048859007+0j)
IZZI	(0.16614543338049342+0j)
IZZZ	(-8.326672684688674e-17+0j)
XXXX	(0.045232799794893426+0j)
XXYY	(0.045232799794893426+0j)
YYXX	(0.045232799794893426+0j)
YYYY	(0.045232799794893426+0j)
ZIII	(-0.2257535025154061+0j)
ZIIZ	(0.16614543338049342+0j)
ZIZI	(0.17464343142442207+0j)
ZZII	(0.12091263358559991+0j)
ZZIZ	(-2.42861286636753e-17+0j)
ZZZI	(-6.938893903907228e-17+0j)
ZZZZ	(-3.122502256758253e-17+0j)

The exact ground state energy using jordan wigner mapping is: -1.8572750766378716

 --- parity ---
IIII	(-0.8105479862760991+0j)
IIIZ	(0.17218394273085635+0j)
IIZI	(0.1209126335855999+0j)
IIZZ	(-0.2257535025154061+0j)
IXIX	(0.045232799794893426+0j)
IXZX	(-0.045232799794893426+0j)
IZII	(-6.938893903907228e-17+0j)
IZIZ	(0.16614543338049345+0j)
IZZI	(0.17218394273085635+0j)
IZZZ	(0.1689275

Now we run on quantum backend, in this case a simulator

In [14]:
for qubit_op, name in qubit_ops:
    # setup VQE 
    # setup optimizer, use L_BFGS_B optimizer for example
    lbfgs = L_BFGS_B(maxfun=1000, factr=10, iprint=10)

    # setup variational form generator (generate trial circuits for VQE)
    var_form = RY(qubit_op.num_qubits, 5, entanglement='full')

    # setup VQE with operator, variational form, and optimizer
    vqe_algorithm = VQE(qubit_op, var_form, lbfgs, 'matrix')

    backend = BasicAer.get_backend('statevector_simulator')
    quantum_instance = QuantumInstance(backend, pass_manager=PassManager())

    results = vqe_algorithm.run(quantum_instance)

    print("Ground state energy using {}: {}".format(name, results['eigvals'][0]))

Ground state energy using jordan wigner: -1.8570893208672647
Ground state energy using parity: -1.8572686760592785
Ground state energy using bravyi-kitaev: -1.85727507635405
